# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState
import random

player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=random.randint(0, 2),
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="던전을 돌면 뭐가 좋다 했지?")],    
    "dungenon_player": player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
await graph.ainvoke({
    "messages":[add_human_message("스킬 사용법이 뭐야?")]
}, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("내 아이템중에 가장 쎈 무기가 뭐야?")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:566: SyntaxWarning: invalid escape sequence '\-'
  "info": """
/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:660: SyntaxWarning: invalid escape sequence '\-'
  "info": """
/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:749: SyntaxWarning: invalid escape sequence '\+'
  "info": """
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydan

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [], 'room_id': 3, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [101, 102], 'heroine_ids': [1, 2]} balanced_map={'dungeon_id': 2, 'floor_count': 2, 'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [2], 'room_id': 3, 'room

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T03:39:30.878250'}, response_metadata={}, id='c8825bf2-1ed3-45e6-9715-a37704788579'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 상담이나 대화를 할 수 있고, 성장도 시켜줄 수 있어!', additional_kwargs={'created_at': '2025-12-07T03:39:32.367588', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='ee5fe5a6-07cd-4ae2-98ca-2078b4029323'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:32.368887'}, response_metadata={}, id='2dcc1414-3238-4bee-9978-e09d2a0c8ab7'),
 AIMessage(content='스킬은 E와 R 키로 사용할 수 있어! E는 무기 스킬이고, R은 직업 고유 스킬이야!', additional_kwargs={'created_at': '2025-12-07T03:39:33.491351', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='3f1b7289-6e5d-4a2c-8e80-f453ff33cd03'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:33.492573'}, response_metadata={}, id='f249cf4c-e217-4e8f-b653-ba9a1dedaf00'),

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [2]:
await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?")}, config=config
)
player = DungeonPlayerState(
    playerId=101,
    heroineId=1,
    currRoomId=1,
    difficulty=random.randint(0, 2),
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="내가 잡을 수 있지?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.MONSTER_GUIDE: 'MONSTER_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [], 'room_id': 3, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [101, 102], 'heroine_ids': [1, 2]} balanced_map={'dungeon_id': 2, 'floor_count': 2, 'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [2], 'room_id': 3, '

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T03:39:30.878250'}, response_metadata={}, id='c8825bf2-1ed3-45e6-9715-a37704788579'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 상담이나 대화를 할 수 있고, 성장도 시켜줄 수 있어!', additional_kwargs={'created_at': '2025-12-07T03:39:32.367588', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='ee5fe5a6-07cd-4ae2-98ca-2078b4029323'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:32.368887'}, response_metadata={}, id='2dcc1414-3238-4bee-9978-e09d2a0c8ab7'),
 AIMessage(content='스킬은 E와 R 키로 사용할 수 있어! E는 무기 스킬이고, R은 직업 고유 스킬이야!', additional_kwargs={'created_at': '2025-12-07T03:39:33.491351', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='3f1b7289-6e5d-4a2c-8e80-f453ff33cd03'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:33.492573'}, response_metadata={}, id='f249cf4c-e217-4e8f-b653-ba9a1dedaf00'),

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [3]:
result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
    },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [], 'room_id': 3, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [101, 102], 'heroine_ids': [1, 2]} balanced_map={'dungeon_id': 2, 'floor_count': 2, 'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [2], 'room_id': 3, 'room

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T03:39:30.878250'}, response_metadata={}, id='c8825bf2-1ed3-45e6-9715-a37704788579'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 상담이나 대화를 할 수 있고, 성장도 시켜줄 수 있어!', additional_kwargs={'created_at': '2025-12-07T03:39:32.367588', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='ee5fe5a6-07cd-4ae2-98ca-2078b4029323'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:32.368887'}, response_metadata={}, id='2dcc1414-3238-4bee-9978-e09d2a0c8ab7'),
 AIMessage(content='스킬은 E와 R 키로 사용할 수 있어! E는 무기 스킬이고, R은 직업 고유 스킬이야!', additional_kwargs={'created_at': '2025-12-07T03:39:33.491351', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='3f1b7289-6e5d-4a2c-8e80-f453ff33cd03'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:33.492573'}, response_metadata={}, id='f249cf4c-e217-4e8f-b653-ba9a1dedaf00'),

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [4]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="다음 방은 뭐가 나올까?") },
    config=config,
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>]
던전 로우 raw_map={'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [], 'room_id': 3, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [101, 102], 'heroine_ids': [1, 2]} balanced_map={'dungeon_id': 2, 'floor_count': 2, 'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [2], 'room_i

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T03:39:30.878250'}, response_metadata={}, id='c8825bf2-1ed3-45e6-9715-a37704788579'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 상담이나 대화를 할 수 있고, 성장도 시켜줄 수 있어!', additional_kwargs={'created_at': '2025-12-07T03:39:32.367588', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='ee5fe5a6-07cd-4ae2-98ca-2078b4029323'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:32.368887'}, response_metadata={}, id='2dcc1414-3238-4bee-9978-e09d2a0c8ab7'),
 AIMessage(content='스킬은 E와 R 키로 사용할 수 있어! E는 무기 스킬이고, R은 직업 고유 스킬이야!', additional_kwargs={'created_at': '2025-12-07T03:39:33.491351', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='3f1b7289-6e5d-4a2c-8e80-f453ff33cd03'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:33.492573'}, response_metadata={}, id='f249cf4c-e217-4e8f-b653-ba9a1dedaf00'),

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [5]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?") },
    config=config
)
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [], 'room_id': 3, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [101, 102], 'heroine_ids': [1, 2]} balanced_map={'dungeon_id': 2, 'floor_count': 2, 'rooms': [{'size': 1, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 1, 'neighbors': [0, 2], 'monsters': [], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 1, 'neighbors': [1, 3], 'monsters': [], 'room_id': 2, 'room_type': 'event', 'event_type': 1}, {'size': 1, 'neighbors': [2], 'monsters': [2], 'room_id': 3, 'room

[HumanMessage(content='던전을 돌면 뭐가 좋다 했지?', additional_kwargs={'created_at': '2025-12-07T03:39:30.878250'}, response_metadata={}, id='c8825bf2-1ed3-45e6-9715-a37704788579'),
 AIMessage(content='던전 탐사를 하면 경험치와 기억의 조각을 얻을 수 있어! 그걸로 상담이나 대화를 할 수 있고, 성장도 시켜줄 수 있어!', additional_kwargs={'created_at': '2025-12-07T03:39:32.367588', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='ee5fe5a6-07cd-4ae2-98ca-2078b4029323'),
 HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:32.368887'}, response_metadata={}, id='2dcc1414-3238-4bee-9978-e09d2a0c8ab7'),
 AIMessage(content='스킬은 E와 R 키로 사용할 수 있어! E는 무기 스킬이고, R은 직업 고유 스킬이야!', additional_kwargs={'created_at': '2025-12-07T03:39:33.491351', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='3f1b7289-6e5d-4a2c-8e80-f453ff33cd03'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-07T03:39:33.492573'}, response_metadata={}, id='f249cf4c-e217-4e8f-b653-ba9a1dedaf00'),

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [6]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
await graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는법 힌트좀 줘") },
    config=config
)
result = await graph.ainvoke(
    { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
    config=config
)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


[HumanMessage(content='히로인의 호감도 얻는법 힌트좀 줘', additional_kwargs={'created_at': '2025-12-07T03:39:40.232327'}, response_metadata={}, id='7e4b3d3c-2817-4eec-897a-3192cc10e9e9'),
 AIMessage(content='로코는 따뜻한 열기와 머리 쓰다듬는 걸 좋아해요! 금속 소리나 정직함도 효과적! 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 2967, 'total_tokens': 3386, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 383, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2966, 'text_tokens': 2967, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'ad097138-28f5-08b9-c467-6e914c800ccb', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--2e2615ae-fe09-4158-afa8-58d8fe4a94f8-0', usage_metadata={'input_tokens': 2967, 'output_tokens': 36, 'total_tokens': 3386, 'input_token_d